# Summarizer

Erstellt von: Enes Yilmaz, Dennis Waltemathe, Demir Dolovac

## Einleitung

Das Zusammenfassen von Texten ist eine spannende Herausforderung. Besonders bei politischen Reden, wie bei den Bundestagsdebatten gibt es viele Details, was die Generierung von Zusammenfassungen schwierig macht.

Um dieses Problem zu lösen, haben wir uns einen passenden Datensatz gesucht, welcher die Bundestagsdebatten von 1949 bis 2021 enthält. Allerdings gab es dabei ein Problem und zwar existierten keine Referenzzusammenfassungen, die als Vergleich oder Traingsdaten genutzt werden konnten.
Leider konnten wir auch keinen anderen Datensatz finden, der solche Zusammenfassungen enthält. Deshalb haben wir für einen kleinen zufälligen Teil des Datensatzes, Zusammenfassungen generieren lassen. Diese haben wir in einer neuen CSV-Datei gespeichert, um unser Modell darauf zu trainieren.

Im folgenden Bericht werden wir erklären, welche Arten von Zusammenfassungen es gibt, wie wir die Zusammenfassungen erstellt haben, wie das Modell trainiert wurde und welche Ergebnisse wir erzielt haben.

## Zusammenfassungstypen

Es gibt zwei Arten von Textzusammenfassungen, die **abstraktive** und **extraktive** Zusammenfassung.
Bei der abstraktiven Zusammenfassung wird die Kernaussage eines Textes mit eigenen Worten zusammengefasst. 
Das Modell erstellt also eine Zusammenfassung, die nicht 1:1 aus dem ursprünglichen Text entnommen ist. Diese Methode ähnelt der menschlichen Zusammenfassung.
Im Gegensatz dazu werden beim extraktiven Vorgehen die wichtigsten Sätze aus dem Originaltext extrahiert und in eine logische Reihenfolge gebracht. Es werden also keine neuen Sätze generiert, sondern nur relevante Sätze des Originaltextes übernommen.

Beispiel:

Originaltext:<br>
*„Die Bundestagsdebatten sind ein zentraler Bestandteil der politischen Entscheidungsfindung in Deutschland. Sie ermöglichen es den Abgeordneten, ihre Positionen und über wichtige Gesetzesvorhaben zu diskutieren.“*

Extraktive Zusammenfassung:<br>
*„Bundestagsdebatten sind ein zentraler Bestandteil der politischen Entscheidungsfindung in Deutschland, ihre Positionen und über wichtige Gesetzesvorhaben zu diskutieren.“*

Abstraktive Zusammenfassungen:<br>
*„Bundestagsdebatten dienen der politischen Meinungsbildung und Gesetzesberatung.“*


## Datensatz

Für die Entwicklung und Evaluation unserer Datenmodelle nutzen wir den speeches.csv-Datensatz von Open Discourse [Open Discourse Dokumentation](https://open-discourse.github.io/open-discourse-documentation/1.1.0/index.html)
. Dieser Datensatz enthält sämtliche Redebeiträge, die von 1949 bis 2021 im Deutschen Bundestag gehalten wurden, und bietet eine umfangreiche sowie gut strukturierte Grundlage für die Analyse politischer Sprache.

Der Datensatz umfasst über 900.000 Reden, ergänzt durch Metadaten wie Wahlperiode, Fraktion, Sprecher sowie das genaue Datum der Rede. <br>
Die wichtigsten Merkmale des Datensatzes sind:<br>
Klar definierte Strukturen: Die enthaltenen Metadaten ermöglichen eine gezielte Analyse basierend auf politischen Parteien, Legislaturperioden oder individuellen Abgeordneten.<br>
Öffentlich zugänglich und gut dokumentiert: Open Discourse stellt eine detaillierte Dokumentation bereit, die eine einfache Nutzung und Reproduzierbarkeit wissenschaftlicher Arbeiten sicherstellt.
<br><br>
Wir haben jedoch keine Referenzzusammenfassungen in diesem Datensatz, welche für das Trainieren und Testen des Summarizers benötigt werden. Deshalb zeigen wir im Folgenden, wie wir dieses Problem für einen kleinen Teil der speeches.csv gelöst haben.

Zu erst laden wir die speeches.csv:

In [1]:
import pandas as pd
csv_datei = "data_summarizer/speeches.csv"


df = pd.read_csv(csv_datei)

speech_contents = df["speechContent"]

Nun bereiten wir die Originaltexte etwas auf:

In [3]:
import re
def clean_text(text):
    text = text.replace("\n", " ").strip()  # Zeilenumbrüche entfernen
    text = re.sub(r"\{\d+\}", "", text)  # Entfernt die {0}, {1}, {2}, etc. aus den Texten
    text = text.lower()  # Alles in Kleinbuchstaben umwandeln
    return text

Als nächstes filtern wir alle Texte mit mind. 20 Wörtern und wählen zufällig 4000 Texte, welche wir dann in die neue CSV speichern:

In [ ]:
# Filtert Texte mit mindestens 20 Wörtern
filtered_texts = speech_contents[speech_contents.str.split().str.len() >= 20].apply(clean_text)

# Zufällig 4000 Texte auswählen
random_texts = filtered_texts.sample(n=4000, random_state=42)

# Speichert die gefilterten Texte in eine neue CSV-Datei
random_texts.to_csv("data_summarizer/filtered_speeches.csv", index=False)   #ACHTUNG: Hier wird dann überschrieben, am besten ein neuen Dateinamen eingeben!

In [6]:
df_new = pd.read_csv("data_summarizer/filtered_speeches.csv")

df_new.head(10)

,speechContent
0,herr präsident! meine sehr verehrten damen und...
1,"das ist eine subjektive bewertung, frau kolleg..."
2,"herr kollege czaja, ist ihnen entgangen, daß i..."
3,"herr staatsminister, war es nicht, da die verh..."
4,"aber, herr kollege schiller, wollen sie nicht ..."
5,das wort hat nun der innenminister von badenwü...
6,"herr staatssekretär, ist es eigentlich zulässi..."
7,"frau abgeordnete, in dem bericht finden sie di..."
8,"sie müssen sehen, dass der bundesverkehrswegep..."
9,"herr staatssekretär, im hinblick darauf, daß g..."


In [7]:
print(f"Anzahl der Zeilen in alter CSV: {len(df)}")
print(f"Anzahl der Zeilen in neuer CSV: {len(df_new)}")

Anzahl der Zeilen in alter CSV: 907644
Anzahl der Zeilen in neuer CSV: 4000


Anschließend generieren wir für die 4000 Texte jeweils eine Referenzzusammenfassung, dazu verwenden wir **gpt-4o-mini**:

In [ ]:
import openai
import pandas as pd
from tqdm import tqdm
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

openai.api_key = "KEY"

file_path = "data_summarizer/filtered_speeches.csv" 
df = pd.read_csv(file_path)

def generate_summary_gpt(text):
    """
    Erstellt für jeden Originaltext aus der filtered_speeches.csv eine Referenzzusammenfassung.
    :param text: Der Originaltext
    :return: 
    """
    prompt = f"""Erstelle eine prägnante, abstraktive Zusammenfassung der folgenden Bundestagsrede, die Zusammenfassung soll kürzer sein als der Originaltext:
    
    Originaltext: "{text}"
    
    Zusammenfassung: (Bitte beende die Zusammenfassung in vollständigen Sätzen.)"""
    
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Du bist ein Experte für Textzusammenfassungen."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=250,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"Fehler bei der OpenAI-Anfrage: {e}")
        return "Fehler bei der Generierung"

summaries = []
for text in tqdm(df["speechContent"], desc="Generiere abstraktive Zusammenfassungen mit GPT-4", unit="Text"):
    summaries.append(generate_summary_gpt(str(text)))

# Neue summaries Spalte für die Referenzzusammenfassungen
df["summaries"] = summaries

# Datei speichern
output_file = "data_summarizer/filtered_speeches_with_summaries.csv"
df.to_csv(output_file, index=False)

print(f"Zusammenfassungen gespeichert in '{output_file}'.")


In [15]:
df_speeches_with_summaries =  pd.read_csv("data_summarizer/filtered_speeches_with_summaries.csv")
print(df_speeches_with_summaries.columns)
df_speeches_with_summaries.head()

Index(['speechContent', 'summaries'], dtype='object')


,speechContent,summaries
0,herr präsident! meine sehr verehrten damen und...,In seiner Rede kritisiert der Abgeordnete die ...
1,"das ist eine subjektive bewertung, frau kolleg...","Die Rede betont, dass die Erstellung des Bewer..."
2,"herr kollege czaja, ist ihnen entgangen, daß i...","Der Redner weist Herrn Czaja darauf hin, dass ..."
3,"herr staatsminister, war es nicht, da die verh...","Der Redner fordert den Staatsminister auf, die..."
4,"aber, herr kollege schiller, wollen sie nicht ...","Der Redner weist darauf hin, dass die Prognose..."


## Finetuning eines Modells

Im Folgenden erklären wir, wie wir Modelle gefinetuned haben. Dazu haben wir vorher die **filtered_speeches** in eine **train.csv** und **test.csv** aufgeteilt (75% train, 25% test):

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lade deine CSV-Datei
df = pd.read_csv("data_summarizer/filtered_speeches_with_summaries.csv") 

# Prüfe, ob die relevanten Spalten vorhanden sind
if "speechContent" not in df.columns or "summaries" not in df.columns:
    raise ValueError("Die CSV-Datei muss die Spalten 'speechContent' und 'summaries' enthalten.")

# Teile die Daten auf (75% Training, 25% Test)
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Speichere die neuen Dateien
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

print(f"Trainingsdaten: {len(train_df)} Zeilen")
print(f"Testdaten: {len(test_df)} Zeilen")

Zu erst laden wir die Datensätze und legen die Teilmengen für das Training und Testen fest:

In [13]:
from datasets import load_dataset
import torch
dataset = load_dataset("csv", data_files={"train": "data_summarizer/train.csv", "test": "data_summarizer/test.csv"})
device = "cuda" if torch.cuda.is_available() else "cpu"

Als nächstes laden wir den AutoTokenizer für ein vortrainiertes Modell und definieren eine Funktion, die Redebeiträge (`speechContent`) und Zusammenfassungen (`summaries`) tokenisiert. Dabei werden die Eingaben auf eine maximale Länge gebracht, mit Padding ergänzt und abgeschnitten. Anschließend mappen wir die Funktion auf den gesamten Datensatz, um tokenisierte Trainingsdaten zu erstellen.

Wir möchten das Modell [t5-small](https://huggingface.co/google-t5/t5-small) finetunen, welches bereits für die deutsche Sprache bzw. das Zusammenfassen deutscher Texte optimiert wurde. 


In [14]:
from transformers import AutoTokenizer

model_name = "t5-small"  
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["speechContent"], 
        max_length=512, 
        padding="max_length",  
        truncation=True
    )
    labels = tokenizer(
        examples["summaries"], 
        max_length=150, 
        padding="max_length",  # Auch hier Padding auf die maximale Länge
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

Nun trainieren wir das Modell, dabei definieren wir die Trainingsparameter und erstellen einen Trainer, die das Modell fintuned:

In [16]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    do_train=True,
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_104/3582634350.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.100400,1.725781
2,1.919500,1.549682
3,1.741400,1.480842
4,1.688300,1.445464
5,1.576100,1.422272
6,1.609400,1.410084
7,1.542100,1.402987
8,1.618900,1.400709


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=752, training_loss=1.9296705386740096, metrics={'train_runtime': 473.1722, 'train_samples_per_second': 50.721, 'train_steps_per_second': 1.589, 'total_flos': 3248203235328000.0, 'train_loss': 1.9296705386740096, 'epoch': 8.0})

Anschließend speichern wir das Modell:

In [9]:
model.save_pretrained("models_summarizer/t5_finetuned_abgabe_neu/summarizer_model")  #Neu wurde hinzugefügt, damit das bereits trainierte Modell nicht überschrieben wird
tokenizer.save_pretrained("models_summarizer/t5_finetuned_abgabe_neu/summarizer_model")

('t5_finetuned/summarizer_model/tokenizer_config.json',
 't5_finetuned/summarizer_model/special_tokens_map.json',
 't5_finetuned/summarizer_model/spiece.model',
 't5_finetuned/summarizer_model/added_tokens.json',
 't5_finetuned/summarizer_model/tokenizer.json')

## ROUGE-Score
Zur Bewertung der Modelle haben wir den ROUGE-Score (Recall-Oriented Unterstudy for Gisting Evaluation) verwendet, dies ist eine Metrik, welche zur Bewertung der Qualität von autmatisch generierter Texte wie zum Beispiel Zusammenfassungen verwendet wird. 
Der ROUGE-Score vergleicht die vom Modell erzeugte Zusammenfassung mit einer Referenzzusammenfassung, indem er die Übereinstimmungen von n-Grammen (Wortsequenzen) misst. 
So weist ein höher ROUGE-Score auf eine höhere Ähnlichkeit zwischen der generierten Zusammenfassung und der Referenzzusammenfassung hin.

Um die Qualität der generierten Zusammenfassungen genauer bewerten zu können, gibt es verschiedene ROUGE-Varianten, welche verschiedene Teile der inhaltlichen Übereinstimmung berücksichtigen:

**<u>ROUGE-N</u>** <br>
Diese Variante misst Überlappung von n-Grammen (Sequenzen von n aufeinanderfolgenden Wörtern) zwischen der generierten und Referenzzusammenfassung.
So würde beispielsweise der ROUGE-1 Score die Übereinstimmung einzelner Wörter (Unigramme) messen und der ROUGE-2 Score die Übereinstimmung von Wortpaaren (Bigramme).

Beispiel:<br>
Referenz: *„Die Katze sitzt auf dem Stuhl“*<br>
Generiert: *„Der Hund schläft auf dem Stuhl“*

Die gemeinsamen Unigramme wären hier: [„auf“, „dem“, „Stuhl“] und die gemeinsamen Bigramme wären: [„auf dem“, „dem Stuhl“]

**<u>ROUGE-L</u>** <br>
Eine weitere Variante basiert auf der längsten gemeinsamen Teilsequenz (LCS). Hier wird die längste gemeinsame Teilsequenze der beiden Zusammenfassungen gemessen, dabei müssen die Wörter nicht unbedingt nebeneinander stehen.

Beispiel:<br>
Referenz: *„Das Auto ist schnell und groß.“*<br>
Generiert: *„Das rote Auto fährt schnell.“*

Die längste gemeinsame Teilsequenz wäre hier „Das Auto schnell“.

**<u>ROUGE-S</u>** <br>
Die letzte ROUGE-Score Variante misst die Übereinstimmung von Skip-Bigrammen, also Wortpaaren die in einer Reihenfolge vorkommen, aber nicht direkt nebeneinander stehen, dies ermöglicht es Wortpaare aus dem Referenztext zu identifizieren, die auch in der generierten Zusammenfassung vorkommen, dadurch wird der inhaltliche Kontext eines Textes erfasst, selbst wenn Wörter durch andere Begriffe getrennt ist.

Beispiel: <br>
Referenz: *„Die Katze ist auf der Matte.“* <br>
Generiert: *„Die graue Katze springt herum.“*

Im ROUGE-N würde es keine Überlappungen geben, doch mittels Skip-Bigrammen würden hier „Die Katze“ und „Die graue Katze“ übereinstimmen.

## Berechnung der ROUGE-Scores

Die Berechnung basiert auf den Konzepten der **Recall**, **Precsision** und **F1-Score**.
So wird der Recall durch den Anteil der überlappenden Wörtern aus der Referenz mit der generierten Zusammenfassung und der Gesamtanzahl der n-Gramme der Referenzzusammenfassung berechnet:

$$recall = \frac{gemeinsame\ n-Gramme}{n-Gramme\ in\ der\ Referenzzusammenfassung} $$

Die Precision wird durch den Anteil der gemeinsamen n-Gramme und mit der Gesamtzahl der n-Gramme der generierten Zusammenfassung berechnet:

$$precision = \frac{gemeinsame\ n-Gramme}{n-Gramme\ in\ der\ generierten\ Zusammenfassung} $$

Um ein Gleichgewicht zwischen Recall und Precision zu finden wird der F1-Score folgendermaßen berechnet:

$$F1 = 2\ x\ \frac{precision\ x\ recall}{precision\ +\ recall} $$





## Finetuned vs. Original

Nun vergleichen wir das originale Modell und das finedtuned Modell, dazu verwenden wir den soeben beschriebenen ROUGE-Score. Dafür laden wir uns zuerst die Metrik runter:

In [1]:
!pip install rouge-score
#https://thepythoncode.com/article/calculate-rouge-score-in-python#setting-up-the-environment-for-python-implementation

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=25025 sha256=140f2d7799bfd8e9f727c3be9a35ecf7053a3a037a378822f3d8af562a70d1fd
  Stored in directory: c:\users\dolov\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Nun berechnen wir den ROUGE-Score, dazu wird ein Text aus der test.csv verwendet und von beiden Modellen zusammengefasst. Anschließend werden beide Zusammenfassungen mit der Referenzzusammenfassung verglichen, was dann die ROUGE-Scores für Presicion, Recall und F1-Score liefert:

In [5]:
from transformers import pipeline
from rouge_score import rouge_scorer
import pandas as pd

df_test_csv = pd.read_csv("data_summarizer/test.csv")


originaltext = df_test_csv["speechContent"][0]
referenztext = df_test_csv["summaries"][0]

# Finegetuntes Modell
fn_summarizer = pipeline("summarization", model="models_summarizer/t5_finetuned_abgabe/summarizer_model", tokenizer="models_summarizer/t5_finetuned_abgabe/summarizer_model")
fn_zusammenfassung = fn_summarizer(originaltext, max_length=150, min_length=50, do_sample=False)[0]["summary_text"]

# Originales Modell
orig_summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")
orig_zusammenfassung = orig_summarizer(originaltext, max_length=150, min_length=50, do_sample=False)[0]["summary_text"]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ROUGE für finegetuntes Modell
fn_rouge = scorer.score(referenztext, fn_zusammenfassung)

# ROUGE für Basismodell
orig_rouge = scorer.score(referenztext, orig_zusammenfassung)

print("Fine-tuned Modell ROUGE Scores:")
print(f"ROUGE-1: {fn_rouge['rouge1']}")
print(f"ROUGE-2: {fn_rouge['rouge2']}")
print(f"ROUGE-L: {fn_rouge['rougeL']}")

print("\nOriginales Modell ROUGE Scores:")
print(f"ROUGE-1: {orig_rouge['rouge1']}")
print(f"ROUGE-2: {orig_rouge['rouge2']}")
print(f"ROUGE-L: {orig_rouge['rougeL']}")

print()
print(f"Originaltext: {originaltext}\n")
print(f"Referenzzusammenfassung: {referenztext}\n")
print(f"Originales Modell: {orig_zusammenfassung}\n")
print(f"FN-Modell: {fn_zusammenfassung}")

Device set to use cpu
Your max_length is set to 150, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Device set to use cpu
Your max_length is set to 150, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


Fine-tuned Modell ROUGE Scores:
ROUGE-1: Score(precision=0.6774193548387096, recall=0.7777777777777778, fmeasure=0.7241379310344828)
ROUGE-2: Score(precision=0.4666666666666667, recall=0.5384615384615384, fmeasure=0.5)
ROUGE-L: Score(precision=0.5806451612903226, recall=0.6666666666666666, fmeasure=0.6206896551724138)

Originales Modell ROUGE Scores:
ROUGE-1: Score(precision=0.6176470588235294, recall=0.7777777777777778, fmeasure=0.6885245901639345)
ROUGE-2: Score(precision=0.45454545454545453, recall=0.5769230769230769, fmeasure=0.5084745762711863)
ROUGE-L: Score(precision=0.5294117647058824, recall=0.6666666666666666, fmeasure=0.5901639344262295)

Originaltext: herr kollege berger, meine antwort ist kurz; sie lautet nein. die dislozierung der luftlandetruppen des warschauer paktes ist nach dem erkenntnisstand der bundesregierung nach wie vor unverändert. es gibt auch keine anzeichen für mögliche veränderungen außerhalb des mbfr-reduzierungsraums.

Referenzzusammenfassung: Die Bundesr



**Vergleich mit anderen Modellen:**

Dazu werden die finegetunten Modelle mit den originalen Modellen vergleichen. Die finegetunten Modelle sind unter (`mt5-small/summarizer_model`) (`t5_finetuned_abgabe/summarizer_model`) und (`bert_finetuned/summarizer_model`) zu finden.

Originaltext: df_test_csv["speechContent"][622]

<div style="display: flex; justify-content: space-around;">

<!-- Erste Tabelle -->
<div>
<p style="text-align: center;"><strong>Finetuned Modelle</strong></p>
<table>
<tr>
<th>ROUGE-SCORE</th><th>mt5-small</th><th>t5-small</th><th>mbart-large-50</th>
</tr>
<tr><td>R1-Precision</td><td>0.4848</td><td>0.6</td><td>0.7096</td></tr>
<tr><td>R1-Recall</td><td>0.5925</td><td>0.5555</td><td>0.4313</td></tr>
<tr><td>R1-F1</td><td>0.5333</td><td>0.5769</td><td>0.5365</td></tr>
<tr><td>R2-Precision</td><td>0.3125</td><td>0.3333</td><td>0.3666</td></tr>
<tr><td>R2-Recall</td><td>0.3846</td><td>0.3076</td><td>0.22</td></tr>
<tr><td>R2-F1</td><td>0.3448</td><td>0.32</td><td>0.2749</td></tr>
<tr><td>RL-Precision</td><td>0.3636</td><td>0.32</td><td>0.6129</td></tr>
<tr><td>RL-Recall</td><td>0.4444</td><td>0.2962</td><td>0.3725</td></tr>
<tr><td>RL-F1</td><td>0.3999</td><td>0.3076</td><td>0.4634</td></tr>
</table>
</div>

<!-- Zweite Tabelle -->
<div>
<p style="text-align: center;"><strong>Originale Modelle</strong></p>
<table>
<tr>
<th>ROUGE-SCORE</th><th>mt5-small</th><th>t5-small</th><th>mbart-large-50</th>
</tr>
<tr><td>R1-Precision</td><td>0.35</td><td>0.56</td><td>0.4</td></tr>
<tr><td>R1-Recall</td><td>0.2592</td><td>0.5185</td><td>0.7450</td></tr>
<tr><td>R1-F1</td><td>0.2978</td><td>0.5384</td><td>0.5205</td></tr>
<tr><td>R2-Precision</td><td>0.1578</td><td>0.375</td><td>0.2978</td></tr>
<tr><td>R2-Recall</td><td>0.1153</td><td>0.3461</td><td>0.56</td></tr>
<tr><td>R2-F1</td><td>0.1333</td><td>0.3599</td><td>0.3888</td></tr>
<tr><td>RL-Precision</td><td>0.35</td><td>0.52</td><td>0.3789</td></tr>
<tr><td>RL-Recall</td><td>0.2592</td><td>0.4814</td><td>0.7058</td></tr>
<tr><td>RL-F1</td><td>0.2978</td><td>0.5</td><td>0.4931</td></tr>
</table>
</div>

</div>


**Vergleich mit anderen Modellen:**

Originaltext: df_test_csv["speechContent"][0]<br>

**Was man hier sehen kann ist, dass besonders das optimierte mt5-small Modell um einiges besser ist als das originale mt5-small Modell.**
<div style="display: flex; justify-content: space-around;">

<!-- Erste Tabelle -->
<div>
<p style="text-align: center;"><strong>Finetuned Modelle</strong></p>
<table>
<tr>
<th>ROUGE-SCORE</th><th>mt5-small</th><th>t5-small</th><th>mbart-large-50</th>
</tr>
<tr><td>R1-Precision</td><td>0.6060</td><td>0.6774</td><td>0.6363</td></tr>
<tr><td>R1-Recall</td><td>0.7407</td><td>0.7777</td><td>0.7777</td></tr>
<tr><td>R1-F1</td><td>0.6666</td><td>0.7241</td><td>0.7</td></tr>
<tr><td>R2-Precision</td><td>0.4375</td><td>0.4666</td><td>0.4687</td></tr>
<tr><td>R2-Recall</td><td>0.5384</td><td>0.5384</td><td>0.5769</td></tr>
<tr><td>R2-F1</td><td>0.4827</td><td>0.5</td><td>0.5172</td></tr>
<tr><td>RL-Precision</td><td>0.5151</td><td>0.5806</td><td>0.5757</td></tr>
<tr><td>RL-Recall</td><td>0.6296</td><td>0.6666</td><td>0.7037</td></tr>
<tr><td>RL-F1</td><td>0.5666</td><td>0.6206</td><td>0.6333</td></tr>
</table>
</div>

<!-- Zweite Tabelle -->
<div>
<p style="text-align: center;"><strong>Originale Modelle</strong></p>
<table>
<tr>
<th>ROUGE-SCORE</th><th>mt5-small</th><th>t5-small</th><th>mbart-large-50</th>
</tr>
<tr><td>R1-Precision</td><td>0.1</td><td>0.6176</td><td>0.4772</td></tr>
<tr><td>R1-Recall</td><td>0.1111</td><td>0.7777</td><td>0.7777</td></tr>
<tr><td>R1-F1</td><td>0.1052</td><td>0.6885</td><td>0.5915</td></tr>
<tr><td>R2-Precision</td><td>0.0</td><td>0.4545</td><td>0.3488</td></tr>
<tr><td>R2-Recall</td><td>0.0</td><td>0.5769</td><td>0.5769</td></tr>
<tr><td>R2-F1</td><td>0.0</td><td>0.5084</td><td>0.4347</td></tr>
<tr><td>RL-Precision</td><td>0.1</td><td>0.5294</td><td>0.4090</td></tr>
<tr><td>RL-Recall</td><td>0.1111</td><td>0.6666</td><td>0.6666</td></tr>
<tr><td>RL-F1</td><td>0.1052</td><td>0.5901</td><td>0.5070</td></tr>
</table>
</div>

</div>


Aber nicht immer schneidet das finetuned Modell besser oder viel besser ab:

**df_test_csv["speechContent"][255]**

Fine-tuned Modell ROUGE Scores:<br>
ROUGE-1: Score(precision=0.3333333333333333, recall=0.4074074074074074, fmeasure=0.36666666666666664)<br>
ROUGE-2: Score(precision=0.125, recall=0.15384615384615385, fmeasure=0.13793103448275862)<br>
ROUGE-L: Score(precision=0.24242424242424243, recall=0.2962962962962963, fmeasure=0.26666666666666666)<br>

Originales Modell ROUGE Scores:<br>
ROUGE-1: Score(precision=0.43333333333333335, recall=0.48148148148148145, fmeasure=0.456140350877193)<br>
ROUGE-2: Score(precision=0.13793103448275862, recall=0.15384615384615385, fmeasure=0.14545454545454548)<br>
ROUGE-L: Score(precision=0.26666666666666666, recall=0.2962962962962963, fmeasure=0.28070175438596495)<br>

Originaltext:<br>
ich habe das erst vor wenigen tagen in einem umfangreichen artikel in der zeitung „die welt" gemacht. wir machen das in vielen fachzeitschriften. je nach bedarf wird dies fortgesetzt.

Referenzzusammenfassung:<br>
Zusammenfassung: "Vor kurzem habe ich einen ausführlichen Artikel in der 'Welt' veröffentlicht. Wir publizieren regelmäßig in Fachzeitschriften und setzen dies je nach Bedarf fort."

Originales Modell:<br>
ich habe das erst vor wenigen tagen in einem umfangreichen artikel in der zeitung „die welt" gemacht. je nach bedarf wird dies dies fortgesetzt. ich mache das in vielen fachzeitschriften.

FN-Modell:<br>
in der Zeitung „die welt“ wird ein umfangreiches Artikel erstellt, das in vielen fachzeitschriften gedruckt wird. Die Fortsetzungen werden nach Bedarf gesetzt. Die Zeitschriften werden in der aktuellen Zeitschrift „die Welt“ veröffentlicht.


Mögliche Gründe dafür können sein:
- **Mangel an Daten**: Mit zu wenigen Trainingsdaten kann das Modell nicht ausreichend lernen und hat Schwierigkeiten bei der Generalisierung.
- **Qualität der Referenzzusammenfassungen**: Da die Referenzzusammenfassungen keine menschlichen Zusammenfassungen sind, sondern generierte, könnten sie weniger genau oder verständlich sein, was die Bewertung des Modells beeinträchtigen kann 
- **Overfitting**: Einige Testdaten ähneln vielleicht den Trainingsdaten, und das Modell könnte sich zu stark an diese spezifischen Beispiele anpassen, wodurch die Generalisierungsfähigkeit auf neue, unterschiedliche Daten eingeschränkt wird.

## Bemerkung
Was uns aufgefallen ist, dass unser optimiertes (`t5-small`) Modell auch für einige Texte bessere Zusammenfassungen erstellt als das originale Modell, selbst wenn der Inhalt nicht viel oder garnichts mit dem trainierten Kontext der Datensätze zu tun hat.

*Ein Beispiel verdeutlicht diesen Unterschied:*

Originaltext:<br>
Die Text-Extraction (englisch text extraction auch englisch keyphrase extraction) bzw. Textextrahierung ist eine Methode zur automatischen Zusammenfassung eines Textes mit Hilfe computerlinguistischer Techniken. Dabei werden Teile eines Textes – zum Beispiel Sätze oder ganze Abschnitte – mittels statistischer und/oder heuristischer Methoden bezüglich ihrer Wichtigkeit oder Relevanz bewertet. Diese scores of importance dienen als Grundlage für die Entscheidung, welche Teile ("keyphrases") extrahiert und zu einem kürzeren Text zusammengestellt werden, der dann einen Überblick über die Inhalte des Originaltextes bietet und in der Regel als extract oder abstract bezeichnet wird.

Nach Karen Spärck Jones (1999) haben die mit dieser Methode produzierten Zusammenfassungen den Nachteil, dass sie zumeist wenig kohärent und somit nur schlecht lesbar und unter Umständen sogar unverständlich sind. Andererseits ist diese Methode und ihre Varianten vermutlich einfacher in automatischen Systemen zu modellieren. Beispiele dafür sind die Systeme von Hans Peter Luhn (1959) (Extraktionsalgorithmus nach Luhn) und Edmundson (1969) und die Ansätze von Rath et al. (1961) und Brandow et al. (1995).

Zusammenfassung des ungetunten Modells (T5-Small, pretrained):<br>
„Dabei werden Teile eines Textes mittels statistischer und/oder heuristischer Techniken bewertet. Diese Scores of Importance dienen als Grundlage für die Entscheidung, welche Teile (Keyphrases) extrahiert und zu einem kürzeren Text zusammengestellt werden, der dann einen Überblick über die Inhalte des Originaltextes bietet und in der Regel als Extract oder Abstract bezeichnet wird.“

Zusammenfassung des getunten Modells:<br>
„Text-Extraction (englisch text extraction) bzw. Textextrahierung ist eine Methode zur automatischen Zusammenfassung von Texten mit Hilfe computerlinguistischer Techniken. Teile einer Texte werden mittels statistischer und/oder heuristischer Methoden bewertet, die einen Überblick über die Inhalte des Originaltextes bieten und in der Regel als Extract oder Abstract bezeichnet werden. Nach Karen Spärck Jones (1999) hat die Methode erstellt Zusammenfassungen, die wenig kohärent sind und die Ansätze von Rath et al.“

Der Unterschied ist deutlich:<br> Das vortrainierte Modell produziert eine eher generische und redundante Zusammenfassung, während das getunte Modell die Begriffe präziser einordnet, den Kontext besser versteht und kohärentere Sätze formuliert.


## Vergleich: extr. und abstr. Summarizer

Für die extraktive Textzusammenfassung setzen wir unter anderem die Python-Bibliothek Sumy ein. Die Stärke von Sumy liegt in seiner einfachen Implementierung und der Möglichkeit, unterschiedliche Methoden für extraktive Zusammenfassungen zu testen. Dies macht es zu einem idealen Werkzeug für unsere Analyse von Redebeiträgen aus dem Open-Discourse-Datensatz.

In [10]:
text = speech_contents[609]

Als Nächstes laden wir die `punkt`-Ressource von **NLTK**, die für die Tokenisierung von Sätzen und Wörtern benötigt wird.  
Diese erlaubt es uns, Texte in einzelne Wörter oder Sätze zu zerlegen.

In [13]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dolov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dolov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

Als Nächstes verwenden wir **Sumy**, um eine extraktive Zusammenfassung mit dem **LexRank-Algorithmus** zu erstellen.  
LexRank ist ein graphbasiertes Verfahren zur Bestimmung der wichtigsten Sätze in einem Text.

In [6]:
!pip install sumy

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.3 MB 2.8 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/6.3 MB 2.1 MB/s eta 0:00:03
   -------- ------------------------------- 1.3/6.3 MB 2.2 MB/s eta 0:00:03
   ------------- --------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


# Parser und Tokenizer
parser = PlaintextParser.from_string(text, Tokenizer("german"))
summarizer = LexRankSummarizer()

# Zusammenfassung erstellen (z. B. 3 Sätze)
summary = summarizer(parser.document, 3)

extractive_sum=""
for sentence in summary:
    extractive_sum += str(sentence)

Für die abstraktive Textzusammenfassung nutzen wir das vortrainierte Modell t5-small. Die Stärke von T5 liegt in seiner Fähigkeit, den Inhalt eines Textes neu zu formulieren und prägnant wiederzugeben, anstatt lediglich wichtige Sätze auszuwählen. Dies macht es zu einem leistungsfähigen Werkzeug für unsere Analyse von Redebeiträgen aus dem Open-Discourse-Datensatz.

In [15]:
from transformers import pipeline, T5Tokenizer

summarizer = pipeline("summarization", model="models_summarizer/t5_finetuned_abgabe/summarizer_model", tokenizer="models_summarizer/t5_finetuned_abgabe/summarizer_model")

abstractive_sum = summarizer(text, max_length=250, min_length=50, do_sample=False)


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (920 > 512). Running this sequence through the model will result in indexing errors


In [16]:
print("Original text: ", text)
print("Extraktive Zusammenfassung: ", extractive_sum)
print()
print("Abstraktive Zusammenfassung: ", abstractive_sum)

Original text:  Nein; ich, sagte: bei der bekannten Loyalität rechnete ich damit, daß Sie das nicht täten, Herr Präsident.
Aber nun zu dem Thema selbst. Ich möchte darauf aufmerksam machen, daß nach meiner Meinung das Berlin-Thema, das wir hier nun verschiedentlich erörtert haben, bisher etwas zu kasuistisch angefaßt worden ist. Man hat heute vom Herrn Finanzminister eine Zusammenfassung verschiedener Hilfsmaßnahmen gehört, allerdings nicht so erschöpfend - oder sollte es mir entgangen sein? -, daß man von allem, auch zum Beispiel von der Portoabgabe ein zahlenmäßiges Ergebnis hätte hören können. Das ist aber nur die eine Seite. Das betrifft nämlich die Maßnahmen, die zum Schutz und zur Unterstützung der kämpfenden Berliner Bevölkerung beschlossen worden sind. Es bedürfte einmal einer systematischen Zusammenstellung, damit wir übersehen, was bisher insgesamt, wie auch was eus den einzelnen Quellen nach Berlin geflossen ist.
Wichtiger scheint mir die andere Seite zu sein. Wir leiden -- 

## Fazit

Zu Beginn des Projekts hatten wir große Schwierigkeiten und mussten viel recherchieren, um die notwendigen Kenntnisse zu erhalten. Eine der größten Herausforderungen war die Datenbeschaffung, da wir keinen Datensatz finden konnten, der eine Referenzzusammenfassung enthielt. Dadurch mussten wir uns eigene Daten zusammenstellen, was den gesamten Prozess deutlich komplizierter machte. Zudem bemerkten wir, dass die Menge der verfügbaren Daten nicht ausreichte, um die Modelle effektiv nach unseren Vorstellungen zu trainieren. Dies wirkte sich etwas auf die Qualität der Ergebnisse aus, insbesondere da die Qualität der generierten Referenzzusammenfassungen den Erfolg des Fine-Tunings beeinflusste.

Trotz dieser Probleme haben wir im Verlauf des Projekts eine Menge gelernt und wichtige Erfahrungen gesammelt. Besonders im Bereich der Textzusammenfassung, beim Fine-Tuning von Modellen und bei der Aufbereitung von Daten konnten wir unser Wissen erweitern. Obwohl es nicht ganz unseren ursprünglichen Vorstellungen eines deutlich genaueren und präziseren Summarizers entsprach, haben uns die Herausforderungen geholfen, besser zu verstehen wie wichtig gute Datensätze sind, um wirklich gute Ergebnisse zu erzielen.

## Quellen

- [Two minutes NLP — Learn the ROUGE metric by examples](https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499)
- [Understanding BLEU and ROUGE score for NLP evaluation](https://medium.com/@sthanikamsanthosh1994/understanding-bleu-and-rouge-score-for-nlp-evaluation-1ab334ecadcb)
- [Extractive vs. Abstractive Summarization](https://www.prodigaltech.com/blog/extractive-vs-abstractive-summarization-how-does-it-work#foldMain)
- [Fine-tune a pretrained model](https://huggingface.co/docs/transformers/en/training)
- [How to Calculate ROUGE Score in Python](https://thepythoncode.com/article/calculate-rouge-score-in-python#setting-up-the-environment-for-python-implementation)